In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [5]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/Movie Dataset.csv',sep=",", encoding='cp1252')

In [6]:
data.head(5)

,title,genre
0,3 Idiots,Comedy
1,Chhichhore,Comedy
2,Thank God,Comedy
3,Dear Zindagi,Comedy
4,Krazzy 4,Comedy


In [7]:
data.dtypes

title    object
genre    object
dtype: object

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   50 non-null     object
 1   genre   50 non-null     object
dtypes: object(2)
memory usage: 928.0+ bytes


In [9]:
data['genre'] = data['genre'].astype('str')
data['genre'] = data['genre'].str.split() 

In [10]:
data.head(10)

,title,genre
0,3 Idiots,[Comedy]
1,Chhichhore,[Comedy]
2,Thank God,[Comedy]
3,Dear Zindagi,[Comedy]
4,Krazzy 4,[Comedy]
5,Wake up Sid,[Comedy]
6,Don't look up,[Comedy]
7,Mask,[Comedy]
8,Lage raho munna bhai,[Comedy]
9,Munna Bhai MBBS,[Comedy]


In [11]:
s = data.apply(lambda x: pd.Series(x['genre']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'genre'
data=data.drop('genre', axis=1).join(s)

In [12]:
data.head(10)

,title,genre
0,3 Idiots,Comedy
1,Chhichhore,Comedy
2,Thank God,Comedy
3,Dear Zindagi,Comedy
4,Krazzy 4,Comedy
5,Wake up Sid,Comedy
6,Don't look up,Comedy
7,Mask,Comedy
8,Lage raho munna bhai,Comedy
9,Munna Bhai MBBS,Comedy


In [13]:
data['index']=data.index
data.head(10)

,title,genre,index
0,3 Idiots,Comedy,0
1,Chhichhore,Comedy,1
2,Thank God,Comedy,2
3,Dear Zindagi,Comedy,3
4,Krazzy 4,Comedy,4
5,Wake up Sid,Comedy,5
6,Don't look up,Comedy,6
7,Mask,Comedy,7
8,Lage raho munna bhai,Comedy,8
9,Munna Bhai MBBS,Comedy,9


In [14]:
data = pd.DataFrame(data, index=data['index'])
data.reset_index(drop=True,inplace=True)

In [15]:
data.shape

(59, 3)

In [16]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(data['genre'])
tfidf_matrix.shape

(59, 7)

In [17]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [18]:
indices = pd.Series(data.index, index=data['genre'])

In [19]:
def get_recommendations(genre,cosine_sim=cosine_sim,num=10):
    idx=indices[genre]
    sim_scores=list(enumerate(cosine_sim[idx]))
    sim_scores=list(np.argsort(sim_scores[0][1]))
    sim_scores.reverse()
    top_similar=sim_scores[0:num+1]
    book_idx=[i for i in top_similar]
    ans=data[['title']].iloc[book_idx]
    ans=ans.sample(n=10)
    return ans

In [20]:
get_recommendations('Biography')

,title
11,Guru
16,Shershaah
17,pirates of silicon valley
14,Super 30
18,The founder
19,Elvis
12,Sala khadush
15,Mary Kom
13,Manjhi ï¿½ The Mountain Man
10,M.S.dhoni


In [21]:
def get_recommendation(genre):
    df=data[data['genre']==genre]
    ans=df[['title']]
    if(len(ans)<10):
      return ans
    else:
      return ans.sample(n=10)

In [22]:
get_recommendation('Biography')

,title
12,Sala khadush
16,Shershaah
14,Super 30
18,The founder
19,Elvis
13,Manjhi ï¿½ The Mountain Man
17,pirates of silicon valley
10,M.S.dhoni
15,Mary Kom
11,Guru


In [23]:
import pickle
pickle.dump(data,open('/content/drive/My Drive/Colab Notebooks/pkl/movie_list.pkl','wb'))
pickle.dump(cosine_sim,open('/content/drive/My Drive/Colab Notebooks/pkl/cosine_sim_movie.pkl','wb'))